# Compressie en karakterherkenning met PCA

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import tree

Lees de datasets in (een set van 60000 beeldjes om te trainen en 10000 beeldjes om te testen)

In [ ]:
mnist_train = pd.read_csv('../week8_Beslissingsbomen/mnist_train.csv')
mnist_test = pd.read_csv('../week8_Beslissingsbomen/mnist_test.csv')

De kolom 'value' bevat de te voorspellen waarde.  Splits de tabellen:

In [ ]:
data = mnist_train.drop(columns='value')
target = mnist_train['value']

data_test = mnist_test.drop(columns='value')
target_test = mnist_test['value']

We kunnen een paar beeldjes laten zien:

In [ ]:
for i in range(16):
    plt.subplot(4, 4, 1 + i)
    plt.imshow(np.reshape(data.iloc[i].to_list(), (28,28)), cmap=plt.get_cmap('gray'))
plt.show()

We maken eerst een PCA transformatie-matrix:

In [ ]:
model_pca = PCA()
model_pca.fit(data)

Wat is de gecumuleerde verklaarde variantie voor de gevonden principal components?

In [ ]:
print(model_pca.explained_variance_ratio_.cumsum().round(3))

We willen 90% van de data behouden:

In [ ]:
model_pca = PCA(n_components=.9)
compressed = model_pca.fit_transform(data)
print(compressed.shape)

Hoeveel principal components werden er dus weerhouden?

Hoe zien die beeldjes er nu uit?

In [ ]:
decompressed = pd.DataFrame(model_pca.inverse_transform(compressed))
for i in range(16):
    plt.subplot(4, 4, 1 + i)
    plt.imshow(np.reshape(decompressed.iloc[i].to_list(), (28,28)), cmap=plt.get_cmap('gray'))
plt.show()

Wat kunnen we nu bereiken met een beslissingsboom?

In [ ]:
model_tree = tree.DecisionTreeClassifier(random_state=42, criterion="entropy", max_depth=19)
model_tree.fit(compressed, target)

Nu testen we het model met de testset (die moet ook eerst getransformeerd worden):

In [ ]:
transformed_test = model_pca.transform(data_test)
predicted = model_tree.predict(transformed_test)
df = pd.DataFrame({'predicted':predicted, 'real':target_test})
print(df.head(20))

We berekenen de nauwkeurigheid van de voorspelling door te tellen hoeveel procent van de voorspelde waarden juist was:

In [ ]:
accuracy = (df.predicted == df.real).sum() / len(df)
print(accuracy)

Is dit beter of slechter dan de vorige versie zonder compressie?  Is dit sneller dan de vorige versie?